In [1]:
# %pip list

In [1]:
import pandas as pd

In [3]:
# %%bigquery
# SELECT * FROM `bigquery-public-data.san_francisco_trees.street_trees` LIMIT 10

In [4]:
# %%bigquery df_tree
# SELECT species, count(*) as count, min(plant_date) as min_date, max(plant_date) as max_date 
# FROM `bigquery-public-data.san_francisco_trees.street_trees`
# group by 1 LIMIT 10

In [5]:
# df_tree

In [19]:
REGION = 'us-central1'
PROJECT_ID = 'gcp-demo1-362917'
DATANAME = 'fraud'
NOTEBOOK = '01'

BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

In [2]:
from google.cloud import storage
from google.cloud import bigquery
import pandas as pd
from sklearn import datasets

In [13]:
# Create a bucket in storage, bucket is a folder, name is gcp-demo1-362917

BUCKET = PROJECT_ID

gcs = storage.Client(project = PROJECT_ID)

if not gcs.lookup_bucket(BUCKET):
    bucketDef = gcs.bucket(BUCKET)
    bucket = gcs.create_bucket(bucketDef, project = PROJECT_ID, location = REGION)
    print(bucket)
else:
    print(gcs.lookup_bucket(BUCKET))

In [18]:
# storage data in that bucket
# 1. create a client that goes to bigquery
bq = bigquery.Client(project = PROJECT_ID)

# 2. create destination in  bucket gcp-demo1-362917, and create a folder 'fraud', and create sub-folder name 'data', storage fraud.csv
destination = f"gs://{BUCKET}/{DATANAME}/data/{DATANAME}.csv"

# 3. source from BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'
source = bigquery.TableReference.from_string(BQ_SOURCE)

# 4. move the data
extract = bq.extract_table(source, destination)

extract.result()

ExtractJob<project=gcp-demo1-362917, location=US, id=0d01e539-f3d1-4de4-b8ac-073f2dc4e2d8>

In [37]:
# Bigquery is like a data warehouse --> project --> datasets --> tables

# 1. List BigQuery datasets in the project, since just started project, no datsets in project
datasets = list(bq.list_datasets())
print(datasets)
for d in datasets:
    print(d.dataset_id)

# 2. if no datasets there, then create dataset fraud
ds = bigquery.Dataset(f"{PROJECT_ID}.{DATANAME}")
ds.location = REGION
ds.labels = {'experiment': f'{NOTEBOOK}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

# 3. check again
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)


fraud
fraud


In [38]:
# now dataset or schema 'fraud' created, but no tables in there.

# need to get data from google cloud storage, bring to bigquery as table

# table 's destination is: go to big query-->project_id--> schema fraud -->filename

# bigquery.LoadJobConfig can put data from storage to bigquery

# load 

from google.cloud.exceptions import NotFound
try:
    table = bq.get_table(f'{PROJECT_ID}.{DATANAME}.{DATANAME}')
    if table:
        print(f'The table already exists: {PROJECT_ID}.{DATANAME}.{DATANAME}')
except NotFound as error:
    print(f'Creating Table ...')
    destination = bigquery.TableReference.from_string(f"{PROJECT_ID}.{DATANAME}.{DATANAME}")
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True,
        labels = {'experiment': f'{NOTEBOOK}'}
    )
    job = bq.load_table_from_uri(f"gs://{BUCKET}/{DATANAME}/data/{DATANAME}.csv", destination, job_config = job_config)
    job.result()
    print(f'Finished creating table: {PROJECT_ID}.{DATANAME}.{DATANAME}')

Creating Table ...
Finished creating table: gcp-demo1-362917.fraud.fraud


In [39]:
# create dataframe temp
%%bigquery temp

SELECT * FROM `gcp-demo1-362917.fraud.fraud` LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00, 10.00rows/s]


In [40]:
temp.head()
# temp.dtypes

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,282,-0.356466,0.725418,1.971749,0.831343,0.369681,-0.107776,0.751610,-0.120166,-0.420675,...,0.020804,0.424312,-0.015989,0.466754,-0.809962,0.657334,-0.043150,-0.046401,0.0,0
1,380,-1.299837,0.881817,1.452842,-1.293698,-0.025105,-1.170103,0.861610,-0.193934,0.592001,...,-0.272563,-0.360853,0.223911,0.598930,-0.397705,0.637141,0.234872,0.021379,0.0,0
2,820,-0.937481,0.401649,1.882689,-0.362001,0.751088,-0.899262,0.880557,-0.181650,-0.211657,...,-0.001757,0.097379,-0.324050,0.436521,0.509674,0.454116,-0.201804,-0.175439,0.0,0
3,1193,1.130646,0.625391,0.837987,2.506543,-0.107116,-0.245548,0.099603,-0.041457,-0.867319,...,-0.017154,-0.014311,0.086559,0.393496,0.332062,-0.066378,0.013858,0.025382,0.0,0
4,2371,-0.878833,0.133657,2.534047,2.609811,1.510839,2.075778,-0.384729,0.230300,-0.367956,...,-0.296422,-0.255485,-0.583298,-1.677514,0.050524,0.250409,-0.223149,-0.420764,0.0,0


# Review Data

In [3]:
%%bigquery df

SELECT * FROM `gcp-demo1-362917.fraud.fraud`

Downloading: 100%|██████████| 284807/284807 [00:02<00:00, 134927.30rows/s]


In [4]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [5]:
df['Class'].value_counts(normalize = True)

0    0.998273
1    0.001727
Name: Class, dtype: float64

In [6]:
# Create table in bigquery
%%bigquery query

create or replace table fraud.fraud_prepped as 
with add_id as (select *, generate_uuid() as transaction_id from fraud.fraud)

select *,
        CASE
            WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 then 'TRAIN'
            WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 then 'VALIDATE'
            ELSE 'TEST'
        END AS splits
FROM add_id

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 330.99query/s]                         


In [7]:
%%bigquery query

SELECT splits,count(*) as Count, 100*count(*)/(sum(count(*)) OVER()) AS Percentage
FROM fraud.fraud_prepped
GROUP BY splits

Downloading: 100%|██████████| 3/3 [00:01<00:00,  2.52rows/s]


query

# Prediction

In [2]:
REGION = 'us-central1'
PROJECT_ID = 'gcp-demo1-362917'
DATANAME = 'fraud'
NOTEBOOK = '01'

VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id'

In [9]:
from google.cloud import aiplatform
from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

In [10]:
#set up client to get to Vertex end point
aiplatform.init(project = PROJECT_ID, location = REGION)
# to pull data
bigquery = bigquery.Client()

In [11]:
DIR = f"temp/{NOTEBOOK}"

In [16]:
# REMOVE & CREATE NEW Folder temp in local
!rm -rf {DIR}
!mkdir -p {DIR}

In [17]:
%%bigquery pred

select * from fraud.fraud_prepped where splits = 'TEST' LIMIT 10;

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.72rows/s]


In [20]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,118003,-0.273529,1.166143,0.274748,2.645038,1.557185,-0.016051,1.033940,-0.095862,-1.656996,...,-0.029181,-0.803796,0.003611,0.107055,-0.067756,0.001065,0.0,0,ab5caad8-4ddc-4c6f-b6dc-b74983d862e9,TEST
1,68497,1.134746,-0.038303,0.762562,0.719867,-0.520257,0.014614,-0.316670,0.192344,0.296568,...,0.128194,0.232815,0.199703,0.182187,-0.002852,0.003787,0.0,0,63d7a24d-3830-4eac-8edf-1138e6d3a8a4,TEST
2,164711,-0.205769,0.990787,1.613205,2.657047,0.466503,0.767889,0.182299,0.403098,-1.060433,...,-0.067296,-0.390004,-1.071495,-0.137284,0.244139,0.239377,0.0,0,98e8c58d-5f74-4dd0-92be-aab0f555f7d5,TEST
3,23224,0.969471,0.370774,1.144937,2.669094,0.472665,1.965695,-0.541647,0.531607,0.986478,...,0.291498,-1.064150,-0.069401,-0.185429,0.065354,0.010112,0.0,0,24fd4119-6b97-4f46-87a8-335963907496,TEST
